In [1]:
# import pandas library
import pandas as pd

In [2]:
# Import data
df = pd.read_csv('data/kc_house_data.csv')
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,...,7 Average,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,7 Average,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,6 Low Average,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,7 Average,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,8 Good,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [3]:
# Check for missing values
missing_values = df.isnull().sum()
print("Missing values before cleaning:")
print(missing_values)


Missing values before cleaning:
id                  0
date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2376
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3842
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64


In [4]:
# Convert the `date` column to a datetime format
df['date'] = pd.to_datetime(df['date'])

In [5]:
# Handle inconsistencies
categorical_columns = ['waterfront', 'view', 'condition', 'grade']
df[categorical_columns] = df[categorical_columns].astype('category')

In [6]:
# Fill missing values
df['waterfront'] = df['waterfront'].cat.add_categories(['No'])
df['waterfront'].fillna('No', inplace=True)
numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns
for column in numerical_columns:
    df[column].fillna(df[column].median(), inplace=True)
for column in categorical_columns:
    if df[column].isnull().sum() > 0:
        df[column].fillna(df[column].mode()[0], inplace=True)

In [7]:
# Identify and handle outliers in numerical columns using IQR
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

for column in numerical_columns:
    df = remove_outliers(df, column)

In [8]:
# Ensure consistent categories in categorical columns
def clean_categories(df, column):
    df[column] = df[column].str.strip().str.lower()
    df[column] = df[column].replace({
        'yes': 'Yes',
        'no': 'No'
    })
    return df

for column in categorical_columns:
    df = clean_categories(df, column)

In [9]:
# Identify and handle invalid data entries
df = df[(df['sqft_living'] > 0) & (df['price'] > 0)]

In [10]:
# find out if missing values and inconsistencies are handled
missing_values1 = df.isnull().sum()
print("Missing values before cleaning:")
print(missing_values1)

Missing values before cleaning:
id               0
date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64


In [11]:
# Verify the changes

print("Data after cleaning inconsistencies:")
print(df.head())

Data after cleaning inconsistencies:
           id       date     price  bedrooms  bathrooms  sqft_living  \
0  7129300520 2014-10-13  221900.0         3       1.00         1180   
2  5631500400 2015-02-25  180000.0         2       1.00          770   
3  2487200875 2014-12-09  604000.0         4       3.00         1960   
4  1954400510 2015-02-18  510000.0         3       2.00         1680   
6  1321400060 2014-06-27  257500.0         3       2.25         1715   

   sqft_lot  floors waterfront  view  ...          grade sqft_above  \
0      5650     1.0         No  none  ...      7 average       1180   
2     10000     1.0         No  none  ...  6 low average        770   
3      5000     1.0         No  none  ...      7 average       1050   
4      8080     1.0         No  none  ...         8 good       1680   
6      6819     2.0         No  none  ...      7 average       1715   

   sqft_basement yr_built  yr_renovated  zipcode      lat     long  \
0            0.0     1955        

In [12]:
# Save the cleaned dataframe to a new CSV file
cleaned_file_path = './data/cleaned_kc_house_data.csv'
df.to_csv(cleaned_file_path, index=False)

In [13]:
# Display summary of cleaned data
df.describe(include='all')

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,1.575800e+04,15758,1.575800e+04,15758.000000,15758.000000,15758.000000,15758.000000,15758.000000,15758,15758,...,15758,15758.000000,15758,15758.000000,15758.0,15758.000000,15758.000000,15758.000000,15758.000000,15758.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,5,...,8,NaN,216,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,none,...,7 average,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15748,14797,...,7641,NaN,9357,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,4.718217e+09,2014-10-30 22:34:33.461099264,4.476821e+05,3.251999,1.970777,1812.605216,6907.915598,1.455832,NaN,NaN,...,NaN,1555.326945,NaN,1969.859627,0.0,98082.577802,47.560322,-122.239483,1792.545755,6789.563016
min,2.800031e+06,2014-05-02 00:00:00,7.800000e+04,2.000000,0.500000,540.000000,520.000000,1.000000,NaN,NaN,...,NaN,480.000000,NaN,1900.000000,0.0,98001.000000,47.189500,-122.475000,620.000000,651.000000
25%,2.310010e+09,2014-07-23 00:00:00,3.000000e+05,3.000000,1.500000,1340.000000,4668.500000,1.000000,NaN,NaN,...,NaN,1130.000000,NaN,1950.000000,0.0,98033.000000,47.466625,-122.339000,1430.000000,4800.000000
50%,4.037201e+09,2014-10-20 00:00:00,4.145000e+05,3.000000,2.000000,1740.000000,7000.000000,1.000000,NaN,NaN,...,NaN,1420.000000,NaN,1970.000000,0.0,98075.000000,47.571200,-122.275000,1710.000000,7133.500000
75%,7.436500e+09,2015-02-18 00:00:00,5.600000e+05,4.000000,2.500000,2220.000000,8892.000000,2.000000,NaN,NaN,...,NaN,1870.000000,NaN,1996.000000,0.0,98119.000000,47.682200,-122.163000,2100.000000,8633.000000
max,9.900000e+09,2015-05-24 00:00:00,1.120000e+06,5.000000,4.000000,3950.000000,17965.000000,3.500000,NaN,NaN,...,NaN,3200.000000,NaN,2015.000000,0.0,98199.000000,47.777600,-121.878000,3160.000000,14775.000000
